# Privacy Evaluation

In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from geopy import distance

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

### Import Datasets

In [3]:
# queries.csv: location queries issued by simulated users
# (IP address, lat, lng, timestamp, POI type filter)
queries_df = pd.read_csv('queries.csv', sep=' ')

# pois.csv: current database of pois
# (POI id, cell id, poi type, lat, lng)
pois_df = pd.read_csv('pois.csv', sep=' ')

In [4]:
queries_df

,ip_address,lat,lon,timestamp,poi_type_query
0,34.101.177.245,46.532942,6.591174,14.912448,cafeteria
1,34.101.177.245,46.532942,6.591174,14.912448,restaurant
2,34.101.177.245,46.550342,6.602852,18.024657,restaurant
3,34.101.177.245,46.550342,6.602852,18.024657,cafeteria
4,34.101.177.245,46.532942,6.591174,36.334539,cafeteria
...,...,...,...,...,...
20438,11.173.13.2,46.524410,6.625246,449.159554,supermarket
20439,11.173.13.2,46.527363,6.628705,453.426750,supermarket
20440,11.173.13.2,46.527363,6.628705,453.426750,gym
20441,11.173.13.2,46.524410,6.625246,464.420041,supermarket


In [5]:
queries_df.dtypes

ip_address         object
lat               float64
lon               float64
timestamp         float64
poi_type_query     object
dtype: object

In [6]:
# What are the poi_type_query?
queries_df['poi_type_query'].unique()

array(['cafeteria', 'restaurant', 'dojo', 'club', 'bar', 'supermarket',
       'gym'], dtype=object)

In [7]:
pois_df.head()

,poi_id,cell_id,poi_type,lat,lon
0,307,1,bar,46.504486,6.559631
1,331,1,bar,46.500259,6.554721
2,578,1,supermarket,46.506017,6.551165
3,897,1,club,46.504494,6.553889
4,972,1,supermarket,46.502984,6.550519


In [8]:
pois_df.dtypes

poi_id        int64
cell_id       int64
poi_type     object
lat         float64
lon         float64
dtype: object

### Query Results for Obfuscated and Unobfuscated Data

In [9]:
# Responses from queries
p_og = pd.read_csv('perturbed_queries_responses.csv', sep=',')
u_og = pd.read_csv('queries_responses.csv', sep=',')

In [10]:
p_og.head()

,Unnamed: 0,ip_address,lat,lon,timestamp,poi_type_query,response
0,0,34.101.177.245,46.534139,6.589529,14.912448,cafeteria,"Casey LLC/Anderson, Levy and Cox/Harmon, Caldw..."
1,1,34.101.177.245,46.531611,6.591255,14.912448,restaurant,"Kerr LLC/Wilson, Sanders and Clark/Williams an..."
2,2,34.101.177.245,46.548663,6.602218,18.024657,restaurant,"Love, Nunez and Hernandez/Carter-Giles/Osborne..."
3,3,34.101.177.245,46.550243,6.604355,18.024657,cafeteria,"Giles, Johnson and Coleman/Smith Inc/Miles, Le..."
4,4,34.101.177.245,46.534427,6.591820,36.334539,cafeteria,Miller Inc/Miller-Williams/Osborne-Nunez/Rodri...


In [11]:
u_og.head()

,Unnamed: 0,ip_address,lat,lon,timestamp,poi_type_query,response
0,0,34.101.177.245,46.532942,6.591174,14.912448,cafeteria,"Kerr LLC/Wilson, Sanders and Clark/Williams an..."
1,1,34.101.177.245,46.532942,6.591174,14.912448,restaurant,"Kerr LLC/Wilson, Sanders and Clark/Williams an..."
2,2,34.101.177.245,46.550342,6.602852,18.024657,restaurant,Oliver Ltd/Clarke-Price/Harmon-Robertson/Silva...
3,3,34.101.177.245,46.550342,6.602852,18.024657,cafeteria,Oliver Ltd/Clarke-Price/Harmon-Robertson/Silva...
4,4,34.101.177.245,46.532942,6.591174,36.334539,cafeteria,"Kerr LLC/Wilson, Sanders and Clark/Williams an..."


In [12]:
u_og.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20443 entries, 0 to 20442
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      20443 non-null  int64  
 1   ip_address      20443 non-null  object 
 2   lat             20443 non-null  float64
 3   lon             20443 non-null  float64
 4   timestamp       20443 non-null  float64
 5   poi_type_query  20443 non-null  object 
 6   response        18928 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 1.1+ MB


In [13]:
# Change ip_address to string
u_og['ip_address'] = u_og['ip_address'].astype(str)
p_og['ip_address'] = p_og['ip_address'].astype(str)

In [14]:
# Fill null values
u_og['response'].fillna('NONE', inplace = True)
p_og['response'].fillna('NONE', inplace = True)

### Calculate top responses for each ip_address

#### Group by ip_address, concatenate response

In [15]:
u = u_og[['ip_address', 'response']]
p = p_og[['ip_address', 'response']]

In [16]:
u_group = u.groupby(['ip_address'], as_index = False).agg({'response': ' '.join})
p_group = p.groupby(['ip_address'], as_index = False).agg({'response': ' '.join})

In [17]:
u_group.head()

,ip_address,response
0,0.98.248.97,Carroll-Romero/Howard Ltd/Oliver-Baldwin/Bucha...
1,10.229.150.53,"Hamilton Group/Garcia, Nash and Sanders/Allen-..."
2,100.255.65.73,"Lee LLC/Kelley-Carter/Hahn, Turner and Russell..."
3,101.193.212.180,"Oconnor, Johnson and Randolph/Collins, Mcgee a..."
4,103.107.27.105,"Hamilton Group/Garcia, Nash and Sanders/Allen-..."


In [18]:
u_group.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ip_address  200 non-null    object
 1   response    200 non-null    object
dtypes: object(2)
memory usage: 4.7+ KB


#### Count top 5 for each ip_address

In [19]:
def top_results(row):
    '''
    Given: row of df
    Returns: array of top five results
    '''
    response_array = row['response'].split('/')
    counts = Counter(response_array)
    top_five = counts.most_common(5)
    
    result = []
    for res in top_five:
        result.append(res[0])
    
    return result[0], result[1], result[2], result[3], result[4]

In [20]:
# unperturbed
u_top = u_group.copy()

u_top['u_top_five'] = u_top.apply(top_results, axis=1)
u_top = u_top.drop(columns=['response'])
u_top.rename(columns={'ip_address': 'u_ip_address'}, inplace=True)
# u_top[['u_1', 'u_2', 'u_3', 'u_4', 'u_5']] = pd.DataFrame(u_top['top_five'].tolist(), index=df.index)
u_top.head()

,u_ip_address,u_top_five
0,0.98.248.97,"(Smith Group, Chambers PLC, Martinez Inc, Tayl..."
1,10.229.150.53,"(Jackson Group, Wood-Hooper, Barnes, Bates and..."
2,100.255.65.73,"(Herrera, Juarez and Thompson, Flores Ltd, Dow..."
3,101.193.212.180,"(Foster Ltd, Gomez-Hayes, Gilmore Ltd, Thomas,..."
4,103.107.27.105,"(Foster Ltd, Gomez-Hayes, Gilmore Ltd, Thomas,..."


In [21]:
# perturbed
p_top = p_group.copy()
p_top['p_top_five'] = p_top.apply(top_results, axis=1)
p_top = p_top.drop(columns=['response'])
p_top.rename(columns={'ip_address': 'p_ip_address'}, inplace=True)
p_top.head()

,p_ip_address,p_top_five
0,0.98.248.97,"(Smith Group, Chambers PLC, Martinez Inc, Tayl..."
1,10.229.150.53,"(Holden and Sons, Miller and Sons, Friedman-Ru..."
2,100.255.65.73,"(Herrera, Juarez and Thompson, Flores Ltd, Dow..."
3,101.193.212.180,"(Thomas PLC, Wright, Mitchell and Fitzgerald, ..."
4,103.107.27.105,"(Thomas PLC, Murphy, Johnson and Wells, Ponce ..."


#### Join the two arrays

In [22]:
join_df = p_top.join(u_top, lsuffix='u_ip_address', rsuffix='p_ip_address')
join_df = join_df.drop(columns=['u_ip_address'])
join_df.rename(columns={'p_ip_address': 'ip_address'}, inplace=True)
join_df.head()

,ip_address,p_top_five,u_top_five
0,0.98.248.97,"(Smith Group, Chambers PLC, Martinez Inc, Tayl...","(Smith Group, Chambers PLC, Martinez Inc, Tayl..."
1,10.229.150.53,"(Holden and Sons, Miller and Sons, Friedman-Ru...","(Jackson Group, Wood-Hooper, Barnes, Bates and..."
2,100.255.65.73,"(Herrera, Juarez and Thompson, Flores Ltd, Dow...","(Herrera, Juarez and Thompson, Flores Ltd, Dow..."
3,101.193.212.180,"(Thomas PLC, Wright, Mitchell and Fitzgerald, ...","(Foster Ltd, Gomez-Hayes, Gilmore Ltd, Thomas,..."
4,103.107.27.105,"(Thomas PLC, Murphy, Johnson and Wells, Ponce ...","(Foster Ltd, Gomez-Hayes, Gilmore Ltd, Thomas,..."


### Determine the similarity between p_top_five and u_top_five

In [23]:
def similarity(row):
    '''
    Given: row of df
    Returns: percentage of u_top_five in p_top_five
    '''
    p_list = row['p_top_five']
    u_list = row['u_top_five']
    
    counter = 0
    for u in u_list:
        if u in p_list:
            counter += 1
    
    percentage = counter / len(p_list)
    
    return percentage

In [24]:
# Finding percentage
similarity_df = join_df.copy()
similarity_df['percentage'] = similarity_df.apply(similarity, axis=1)
similarity_df.head()

,ip_address,p_top_five,u_top_five,percentage
0,0.98.248.97,"(Smith Group, Chambers PLC, Martinez Inc, Tayl...","(Smith Group, Chambers PLC, Martinez Inc, Tayl...",1.0
1,10.229.150.53,"(Holden and Sons, Miller and Sons, Friedman-Ru...","(Jackson Group, Wood-Hooper, Barnes, Bates and...",0.0
2,100.255.65.73,"(Herrera, Juarez and Thompson, Flores Ltd, Dow...","(Herrera, Juarez and Thompson, Flores Ltd, Dow...",1.0
3,101.193.212.180,"(Thomas PLC, Wright, Mitchell and Fitzgerald, ...","(Foster Ltd, Gomez-Hayes, Gilmore Ltd, Thomas,...",0.0
4,103.107.27.105,"(Thomas PLC, Murphy, Johnson and Wells, Ponce ...","(Foster Ltd, Gomez-Hayes, Gilmore Ltd, Thomas,...",0.0


In [25]:
similarity_df['percentage'].values

array([1. , 0. , 1. , 0. , 0. , 0. , 0.8, 0. , 0. , 1. , 1. , 0. , 1. ,
       0.2, 1. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 1. ,
       1. , 0. , 0. , 0.8, 0.2, 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 1. , 0. , 0. , 0.8, 1. , 1. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 1. , 0. ,
       0. , 1. , 0. , 0.2, 1. , 0. , 0. , 0. , 0.8, 0.8, 0. , 0.6, 0. ,
       0. , 0. , 0. , 0. , 0. , 1. , 0. , 0.8, 0. , 0. , 0.2, 0. , 0.4,
       0. , 0.8, 0. , 0. , 0.2, 1. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 1. , 0. , 1. ,
       0. , 1. , 1. , 0.4, 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       1. , 0.8, 0. , 0.8, 0. , 1. , 0. , 0. , 0. , 1. , 0. , 1. , 1. ,
       0. , 0. , 1. , 0. , 0. , 0. , 0. , 1. , 1. , 0. , 0. , 0. , 0. ,
       0. , 0.2, 0. , 0. , 1. , 1. , 1. , 1. , 0. , 0. , 0.8, 0. , 0. ,
       1. , 1. , 0. , 0. , 0. , 1. , 1. , 1. , 0. , 0. , 0. , 0.

In [26]:
similarity_df['percentage'].values.mean()

0.297

### Measure distance differences between perturbed and unperturbed queries

In [27]:
queries_df

,ip_address,lat,lon,timestamp,poi_type_query
0,34.101.177.245,46.532942,6.591174,14.912448,cafeteria
1,34.101.177.245,46.532942,6.591174,14.912448,restaurant
2,34.101.177.245,46.550342,6.602852,18.024657,restaurant
3,34.101.177.245,46.550342,6.602852,18.024657,cafeteria
4,34.101.177.245,46.532942,6.591174,36.334539,cafeteria
...,...,...,...,...,...
20438,11.173.13.2,46.524410,6.625246,449.159554,supermarket
20439,11.173.13.2,46.527363,6.628705,453.426750,supermarket
20440,11.173.13.2,46.527363,6.628705,453.426750,gym
20441,11.173.13.2,46.524410,6.625246,464.420041,supermarket


In [28]:
p_og

,Unnamed: 0,ip_address,lat,lon,timestamp,poi_type_query,response
0,0,34.101.177.245,46.534139,6.589529,14.912448,cafeteria,"Casey LLC/Anderson, Levy and Cox/Harmon, Caldw..."
1,1,34.101.177.245,46.531611,6.591255,14.912448,restaurant,"Kerr LLC/Wilson, Sanders and Clark/Williams an..."
2,2,34.101.177.245,46.548663,6.602218,18.024657,restaurant,"Love, Nunez and Hernandez/Carter-Giles/Osborne..."
3,3,34.101.177.245,46.550243,6.604355,18.024657,cafeteria,"Giles, Johnson and Coleman/Smith Inc/Miles, Le..."
4,4,34.101.177.245,46.534427,6.591820,36.334539,cafeteria,Miller Inc/Miller-Williams/Osborne-Nunez/Rodri...
...,...,...,...,...,...,...,...
20438,20438,11.173.13.2,46.526296,6.624683,449.159554,supermarket,Nichols Group/Kelly Group/Kelly Ltd/Bell and S...
20439,20439,11.173.13.2,46.528902,6.629257,453.426750,supermarket,"Morgan, Oliver and Williams/Mooney, Burgess an..."
20440,20440,11.173.13.2,46.526520,6.629954,453.426750,gym,"Morgan, Oliver and Williams/Mooney, Burgess an..."
20441,20441,11.173.13.2,46.524515,6.624620,464.420041,supermarket,Nichols Group/Kelly Group/Kelly Ltd/Bell and S...


In [29]:
u_og

,Unnamed: 0,ip_address,lat,lon,timestamp,poi_type_query,response
0,0,34.101.177.245,46.532942,6.591174,14.912448,cafeteria,"Kerr LLC/Wilson, Sanders and Clark/Williams an..."
1,1,34.101.177.245,46.532942,6.591174,14.912448,restaurant,"Kerr LLC/Wilson, Sanders and Clark/Williams an..."
2,2,34.101.177.245,46.550342,6.602852,18.024657,restaurant,Oliver Ltd/Clarke-Price/Harmon-Robertson/Silva...
3,3,34.101.177.245,46.550342,6.602852,18.024657,cafeteria,Oliver Ltd/Clarke-Price/Harmon-Robertson/Silva...
4,4,34.101.177.245,46.532942,6.591174,36.334539,cafeteria,"Kerr LLC/Wilson, Sanders and Clark/Williams an..."
...,...,...,...,...,...,...,...
20438,20438,11.173.13.2,46.524410,6.625246,449.159554,supermarket,Nichols Group/Kelly Group/Kelly Ltd/Bell and S...
20439,20439,11.173.13.2,46.527363,6.628705,453.426750,supermarket,Bradley and Sons/Cruz-Johnston/Norton and Sons...
20440,20440,11.173.13.2,46.527363,6.628705,453.426750,gym,Bradley and Sons/Cruz-Johnston/Norton and Sons...
20441,20441,11.173.13.2,46.524410,6.625246,464.420041,supermarket,Nichols Group/Kelly Group/Kelly Ltd/Bell and S...


In [40]:
q_df = queries_df.loc[:,['lat','lon']].rename(columns={'lat': 'queries_lat', 'lon': 'queries_lon'})
q_df

,queries_lat,queries_lon
0,46.532942,6.591174
1,46.532942,6.591174
2,46.550342,6.602852
3,46.550342,6.602852
4,46.532942,6.591174
...,...,...
20438,46.524410,6.625246
20439,46.527363,6.628705
20440,46.527363,6.628705
20441,46.524410,6.625246


In [41]:
p_df = p_og.loc[:,['lat','lon']].rename(columns={'lat': 'p_lat', 'lon': 'p_lon'})
p_df

,p_lat,p_lon
0,46.534139,6.589529
1,46.531611,6.591255
2,46.548663,6.602218
3,46.550243,6.604355
4,46.534427,6.591820
...,...,...
20438,46.526296,6.624683
20439,46.528902,6.629257
20440,46.526520,6.629954
20441,46.524515,6.624620


In [42]:
u_df = u_og.loc[:,['lat','lon']].rename(columns={'lat': 'u_lat', 'lon': 'u_lon'})
u_df

,u_lat,u_lon
0,46.532942,6.591174
1,46.532942,6.591174
2,46.550342,6.602852
3,46.550342,6.602852
4,46.532942,6.591174
...,...,...
20438,46.524410,6.625246
20439,46.527363,6.628705
20440,46.527363,6.628705
20441,46.524410,6.625246


In [47]:
total_df = q_df
total_df['p_lat'] = p_df['p_lat']
total_df['p_lon'] = p_df['p_lon']
total_df['u_lat'] = u_df['u_lat']
total_df['u_lon'] = u_df['u_lon']

total_df

,queries_lat,queries_lon,p_lat,p_lon,u_lat,u_lon
0,46.532942,6.591174,46.534139,6.589529,46.532942,6.591174
1,46.532942,6.591174,46.531611,6.591255,46.532942,6.591174
2,46.550342,6.602852,46.548663,6.602218,46.550342,6.602852
3,46.550342,6.602852,46.550243,6.604355,46.550342,6.602852
4,46.532942,6.591174,46.534427,6.591820,46.532942,6.591174
...,...,...,...,...,...,...
20438,46.524410,6.625246,46.526296,6.624683,46.524410,6.625246
20439,46.527363,6.628705,46.528902,6.629257,46.527363,6.628705
20440,46.527363,6.628705,46.526520,6.629954,46.527363,6.628705
20441,46.524410,6.625246,46.524515,6.624620,46.524410,6.625246


In [54]:
total_df.dtypes

queries_lat    float64
queries_lon    float64
p_lat          float64
p_lon          float64
u_lat          float64
u_lon          float64
dtype: object

#### Calculate distance between queries and perturbed/unperturbed responses

In [79]:
def distance_calc_p(row):
    '''
    Calculates the distance between two points
    '''
    p1 = (row['queries_lat'], row['queries_lon'])
    p2 = (row['p_lat'], row['p_lon'])
    
    return distance.distance(p1, p2).kilometers

def distance_calc_u(row):
    '''
    Calculates the distance between two points
    '''
    p1 = (row['queries_lat'], row['queries_lon'])
    p2 = (row['u_lat'], row['u_lon'])
    
    return distance.distance(p1, p2).kilometers

In [80]:
total_df['p_dist'] = total_df.apply(distance_calc_p, axis=1)
total_df['u_dist'] = total_df.apply(distance_calc_u, axis=1)

In [81]:
total_df['p_dist'].mean()

0.14487554140839842

Thus, the average distance difference between the unperturbed and perturbed queries is only 0.15 km.

In [83]:
total_df['p_dist'].max()

0.2684600215899644